In [ ]:
#Import the required libraries
!pip install keras-tuner
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import pathlib, os, random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow as tf
from PIL import Image
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import time
import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler


import cv2
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from PIL import Image
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from keras.utils import np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 32.2 MB/s 
     |████████████████████████████████| 1.6 MB 47.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Understanding the files in different data directories
bad_data = os.listdir('/content/drive/MyDrive/Ethiopia_3/Bad')
print("Bad coffee beans: ",bad_data[:10])

good_data = os.listdir('/content/drive/MyDrive/Ethiopia_3/Good')
print("Good coffee beans: ",good_data[:10])

Bad coffee beans:  ['IMG_5979_9.jpg', 'IMG_5980_2.jpg', 'IMG_5982_9.jpg', 'IMG_5981_7.jpg', 'IMG_5984_1.jpg', 'IMG_5984_3.jpg', 'IMG_5983_3.jpg', 'IMG_5982_8.jpg', 'IMG_5982_2.jpg', 'IMG_5979_4.jpg']
Good coffee beans:  ['IMG_5924_5.jpg', 'IMG_5917_8.jpg', 'IMG_5923_4.jpg', 'IMG_5921_7.jpg', 'IMG_5920_1.jpg', 'IMG_5920_5.jpg', 'IMG_5919_5.jpg', 'IMG_5918_3.jpg', 'IMG_5921_4.jpg', 'IMG_5924_4.jpg']


In [ ]:
data = []
labels = []
for img in bad_data:
    try:
        img_read = plt.imread('/content/drive/MyDrive/Ethiopia_3/Bad' + "/" + img)
        img_resize = cv2.resize(img_read, (256,256))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(1)
    except:
        None
        
for img in good_data:
    try:
        img_read = plt.imread('/content/drive/MyDrive/Ethiopia_3/Good' + "/" + img)
        img_resize = cv2.resize(img_read, (256,256))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(0)
    except:
        None

In [ ]:
image_data = np.array(data)
labels = np.array(labels)

idx = np.arange(image_data.shape[0])
np.random.shuffle(idx)
image_data = image_data[idx]
labels = labels[idx]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size = 0.2, random_state = 101)

y_train = np_utils.to_categorical(y_train, num_classes = 2)
y_test = np_utils.to_categorical(y_test, num_classes = 2)

print(f'SHAPE OF TRAINING IMAGE DATA : {x_train.shape}')
print(f'SHAPE OF TESTING IMAGE DATA : {x_test.shape}')
print(f'SHAPE OF TRAINING LABELS : {y_train.shape}')
print(f'SHAPE OF TESTING LABELS : {y_test.shape}')

SHAPE OF TRAINING IMAGE DATA : (112, 256, 256, 3)
SHAPE OF TESTING IMAGE DATA : (28, 256, 256, 3)
SHAPE OF TRAINING LABELS : (112, 2)
SHAPE OF TESTING LABELS : (28, 2)


In [ ]:
!pip install hyperactive==0.3.1
import hyperactive as hyp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
search_config = {
    "keras.compile.0": {"loss": ["binary_crossentropy"], "optimizer": ["adam"]},
    "keras.fit.0": {"epochs": [5], "batch_size": [32], "verbose": [2]},
    "keras.layers.Conv2D.1": {
        "filters": [32, 64, 128],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
        "input_shape": [(256,256,3)],
    },
    "keras.layers.MaxPooling2D.2": {"pool_size": [(2, 2)]},
    "keras.layers.Conv2D.3": {
        "filters": [16, 32, 64, 128],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
    },
    "keras.layers.MaxPooling2D.4": {"pool_size": [(2, 2)]},
    "keras.layers.Flatten.5": {},
    "keras.layers.Dense.6": {"units": range(30, 100, 10), "activation": ["relu"]},
    "keras.layers.Dropout.7": {"rate": list(np.arange(0.2, 0.8, 0.2))},
    "keras.layers.Dense.8": {"units": [2], "activation": ["softmax"]},
}
start_point = {
    "keras.compile.0": {"loss": ["binary_crossentropy"], "optimizer": ["adam"]},
    "keras.fit.0": {"epochs": [5], "batch_size": [32], "verbose": [2]},
    "keras.layers.Conv2D.1": {
        "filters": [64],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
        "input_shape": [(256,256,3)],
    },
    "keras.layers.MaxPooling2D.2": {"pool_size": [(2, 2)]},
    "keras.layers.Conv2D.3": {
        "filters": [32],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
    },
    "keras.layers.MaxPooling2D.4": {"pool_size": [(2, 2)]},
    "keras.layers.Flatten.5": {},
    "keras.layers.Dense.6": {"units": [30], "activation": ["relu"]},
    "keras.layers.Dropout.7": {"rate": [0.2]},
    "keras.layers.Dense.8": {"units": [2], "activation": ["softmax"]},
}


#https://pypi.org/project/hyperactive/0.3.1/ 
Optimizer = hyp.ParticleSwarmOptimizer(search_config, n_iter=3,warm_start=start_point, metric="accuracy", n_jobs=1, cv=3, verbosity=1, random_state=42, n_part=3, w=0.85, c_k=1.9, c_s=2.05)

In [ ]:
print(len(y_train))
print(y_train[5])

112
[1. 0.]


In [ ]:
partition_size = len(image_data)/5
ctr=0
l1=[]
l2=[]
Final_data=[]
Final_label=[]
for i in range(len(image_data)):
  if ctr==partition_size:
    ctr=0
    Final_data.append(l1)
    Final_label.append(l2)
    l1=[]
    l2=[]
  else:
    l1.append(image_data[i])
    l2.append(labels[i])
    ctr=ctr+1
Final_data.append(l1)
Final_label.append(l2)

In [ ]:
import time
#for i in range(len(Final_data)):
def buildmodel():
  t1 = time.time()
  #x_train=[]
  #y_train=[]
  #x_test=Final_data[i];
  #y_test=Final_label[i] 
  """for j in range(len(Final_data)):
    if j is not i:
      x_train.extend(Final_data[j])
      y_train.extend(Final_label[j])
  
  """
  # search best hyperparameter for given data
  h = Optimizer.fit(x_train, y_train)

  t2 = time.time()
  print("time: {}".format(t2-t1))

  # predict from test data
  Optimizer.predict(x_test)

  # calculate accuracy score
  score = Optimizer.score(x_test, y_test)

  print("test score: {}".format(score))

In [ ]:
import time
t1 = time.time()
# search best hyperparameter for given data
h = Optimizer.fit(x_train, y_train)

t2 = time.time()
print("time: {}".format(t2-t1))

# predict from test data
Optimizer.predict(x_test)

# calculate accuracy score
score = Optimizer.score(x_test, y_test)

print("test score: {}".format(score))

Epoch 1/5
4/4 - 17s - loss: 521.4791 - accuracy: 0.5000 - 17s/epoch - 4s/step
Epoch 2/5
4/4 - 0s - loss: 13.7050 - accuracy: 0.5804 - 255ms/epoch - 64ms/step
Epoch 3/5
4/4 - 0s - loss: 0.4913 - accuracy: 0.8482 - 258ms/epoch - 65ms/step
Epoch 4/5
4/4 - 0s - loss: 0.5225 - accuracy: 0.7857 - 258ms/epoch - 64ms/step
Epoch 5/5
4/4 - 0s - loss: 0.5632 - accuracy: 0.7946 - 252ms/epoch - 63ms/step
4/4 [==============================] - 0s 29ms/step - loss: 6.8005 - accuracy: 0.5000


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/5
4/4 - 1s - loss: 1040.1041 - accuracy: 0.5982 - 1s/epoch - 318ms/step
Epoch 2/5
4/4 - 0s - loss: 154.2354 - accuracy: 0.5179 - 149ms/epoch - 37ms/step
Epoch 3/5
4/4 - 0s - loss: 11.4438 - accuracy: 0.5000 - 152ms/epoch - 38ms/step
Epoch 4/5
4/4 - 0s - loss: 16.3542 - accuracy: 0.4732 - 152ms/epoch - 38ms/step
Epoch 5/5
4/4 - 0s - loss: 9.4658 - accuracy: 0.5089 - 145ms/epoch - 36ms/step
4/4 [==============================] - 0s 19ms/step - loss: 0.8651 - accuracy: 0.5000
Epoch 1/5
4/4 - 1s - loss: 450.2370 - accuracy: 0.5446 - 1s/epoch - 300ms/step
Epoch 2/5
4/4 - 0s - loss: 13.4421 - accuracy: 0.4464 - 240ms/epoch - 60ms/step
Epoch 3/5
4/4 - 0s - loss: 1.5049 - accuracy: 0.6429 - 237ms/epoch - 59ms/step
Epoch 4/5
4/4 - 0s - loss: 0.5967 - accuracy: 0.6875 - 240ms/epoch - 60ms/step
Epoch 5/5
4/4 - 0s - loss: 0.6295 - accuracy: 0.7768 - 228ms/epoch - 57ms/step
4/4 [==============================] - 0s 28ms/step - loss: 0.6418 - accuracy: 0.7946
Epoch 1/5
4/4 - 3s - loss: 328.6

 33%|███▎      | 1/3 [00:12<00:24, 12.09s/it]

Epoch 1/5
4/4 - 1s - loss: 734.1766 - accuracy: 0.5089 - 642ms/epoch - 161ms/step
Epoch 2/5
4/4 - 0s - loss: 187.8229 - accuracy: 0.4554 - 153ms/epoch - 38ms/step
Epoch 3/5
4/4 - 0s - loss: 18.9713 - accuracy: 0.4911 - 151ms/epoch - 38ms/step
Epoch 4/5
4/4 - 0s - loss: 0.9515 - accuracy: 0.7232 - 154ms/epoch - 38ms/step
Epoch 5/5
4/4 - 0s - loss: 0.6166 - accuracy: 0.8125 - 144ms/epoch - 36ms/step
4/4 [==============================] - 0s 19ms/step - loss: 0.6593 - accuracy: 0.8661
Epoch 1/5
4/4 - 1s - loss: 120.8432 - accuracy: 0.5446 - 713ms/epoch - 178ms/step
Epoch 2/5
4/4 - 0s - loss: 5.4052 - accuracy: 0.6964 - 235ms/epoch - 59ms/step
Epoch 3/5
4/4 - 0s - loss: 1.2600 - accuracy: 0.5446 - 237ms/epoch - 59ms/step
Epoch 4/5
4/4 - 0s - loss: 0.6927 - accuracy: 0.4821 - 237ms/epoch - 59ms/step
Epoch 5/5
4/4 - 0s - loss: 0.6932 - accuracy: 0.5357 - 232ms/epoch - 58ms/step
4/4 [==============================] - 0s 28ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 1/5
4/4 - 1s - loss: 50

 67%|██████▋   | 2/3 [00:18<00:08,  8.85s/it]

Epoch 1/5
4/4 - 1s - loss: 401.1234 - accuracy: 0.4643 - 620ms/epoch - 155ms/step
Epoch 2/5
4/4 - 0s - loss: 87.0773 - accuracy: 0.5000 - 148ms/epoch - 37ms/step
Epoch 3/5
4/4 - 0s - loss: 0.8083 - accuracy: 0.4732 - 148ms/epoch - 37ms/step
Epoch 4/5
4/4 - 0s - loss: 0.6995 - accuracy: 0.4911 - 149ms/epoch - 37ms/step
Epoch 5/5
4/4 - 0s - loss: 0.6931 - accuracy: 0.5000 - 139ms/epoch - 35ms/step
4/4 [==============================] - 0s 19ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 1/5
4/4 - 1s - loss: 450.4927 - accuracy: 0.4464 - 657ms/epoch - 164ms/step
Epoch 2/5
4/4 - 0s - loss: 47.4480 - accuracy: 0.4643 - 148ms/epoch - 37ms/step
Epoch 3/5
4/4 - 0s - loss: 4.1071 - accuracy: 0.5714 - 149ms/epoch - 37ms/step
Epoch 4/5
4/4 - 0s - loss: 0.5775 - accuracy: 0.6250 - 148ms/epoch - 37ms/step
Epoch 5/5
4/4 - 0s - loss: 0.4802 - accuracy: 0.7589 - 142ms/epoch - 35ms/step
4/4 [==============================] - 0s 19ms/step - loss: 0.4100 - accuracy: 0.8839
Epoch 1/5
4/4 - 1s - loss: 729


 accuracy 0.9732142686843872
start_point = {'keras.compile.0': {'loss': ['binary_crossentropy'], 'optimizer': ['adam']}, 'keras.fit.0': {'epochs': [5], 'batch_size': [32], 'verbose': [2]}, 'keras.layers.Conv2D.1': {'filters': [32], 'kernel_size': [(3, 3)], 'activation': ['relu'], 'input_shape': [(256, 256, 3)]}, 'keras.layers.MaxPooling2D.2': {'pool_size': [(2, 2)]}, 'keras.layers.Conv2D.3': {'filters': [16], 'kernel_size': [(3, 3)], 'activation': ['relu']}, 'keras.layers.MaxPooling2D.4': {'pool_size': [(2, 2)]}, 'keras.layers.Flatten.5': {}, 'keras.layers.Dense.6': {'units': [40], 'activation': ['relu']}, 'keras.layers.Dropout.7': {'rate': [0.4]}, 'keras.layers.Dense.8': {'units': [2], 'activation': ['softmax']}}
1/4 [======>.......................] - ETA: 0s - loss: 6.3930 - accuracy: 0.4688

4/4 [==============================] - 0s 63ms/step - loss: 2.0744 - accuracy: 0.7857
time: 47.61406207084656
1/1 [==============================] - 0s 50ms/step - loss: 0.6456 - accuracy: 0.7143
test score: 0.7142857313156128


In [ ]:
# KFOLD CROSS VALIDATION

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
kf.get_n_splits(image_data)

5

In [ ]:
image_data = np.array(image_data)
labels = np.array(labels)



for train_index,test_index in kf.split(image_data):
  X_train, X_test = image_data[train_index], image_data[test_index]
  y_train, y_test = labels[train_index], labels[test_index]
  print(X_train.shape,X_test.shape)

  #y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
  #y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
  y_train = np_utils.to_categorical(y_train, num_classes = 2)
  y_test = np_utils.to_categorical(y_test, num_classes = 2)
  t1 = time.time()
# search best hyperparameter for given data
  h = Optimizer.fit(X_train, y_train)

  t2 = time.time()
  print("time: {}".format(t2-t1))

  # predict from test data
  Optimizer.predict(X_test)

  # calculate accuracy score
  score = Optimizer.score(X_test, y_test)

  print("Test score: {}".format(score))

(112, 256, 256, 3) (28, 256, 256, 3)
Epoch 1/5
4/4 - 1s - loss: 412.5158 - accuracy: 0.5179 - 809ms/epoch - 202ms/step
Epoch 2/5
4/4 - 0s - loss: 55.5989 - accuracy: 0.5268 - 256ms/epoch - 64ms/step
Epoch 3/5
4/4 - 0s - loss: 1.2626 - accuracy: 0.5446 - 264ms/epoch - 66ms/step
Epoch 4/5
4/4 - 0s - loss: 1.0189 - accuracy: 0.6071 - 258ms/epoch - 64ms/step
Epoch 5/5
4/4 - 0s - loss: 0.6691 - accuracy: 0.7411 - 251ms/epoch - 63ms/step
4/4 [==============================] - 0s 30ms/step - loss: 0.6059 - accuracy: 0.8482


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/5
4/4 - 1s - loss: 858.8124 - accuracy: 0.5000 - 651ms/epoch - 163ms/step
Epoch 2/5
4/4 - 0s - loss: 64.7830 - accuracy: 0.5625 - 155ms/epoch - 39ms/step
Epoch 3/5
4/4 - 0s - loss: 21.8753 - accuracy: 0.5000 - 151ms/epoch - 38ms/step
Epoch 4/5
4/4 - 0s - loss: 2.8573 - accuracy: 0.5179 - 149ms/epoch - 37ms/step
Epoch 5/5
4/4 - 0s - loss: 3.1364 - accuracy: 0.5625 - 149ms/epoch - 37ms/step
4/4 [==============================] - 0s 19ms/step - loss: 0.6610 - accuracy: 0.7143
Epoch 1/5
4/4 - 1s - loss: 88.1647 - accuracy: 0.4554 - 728ms/epoch - 182ms/step
Epoch 2/5
4/4 - 0s - loss: 4.3626 - accuracy: 0.5625 - 240ms/epoch - 60ms/step
Epoch 3/5
4/4 - 0s - loss: 0.6506 - accuracy: 0.6250 - 239ms/epoch - 60ms/step
Epoch 4/5
4/4 - 0s - loss: 0.5481 - accuracy: 0.6875 - 239ms/epoch - 60ms/step
Epoch 5/5
4/4 - 0s - loss: 0.5101 - accuracy: 0.7589 - 232ms/epoch - 58ms/step
4/4 [==============================] - 0s 28ms/step - loss: 0.4442 - accuracy: 0.8750
Epoch 1/5
4/4 - 1s - loss: 412.

 33%|███▎      | 1/3 [00:07<00:15,  7.52s/it]

Epoch 1/5
4/4 - 1s - loss: 256.6010 - accuracy: 0.5446 - 654ms/epoch - 164ms/step
Epoch 2/5
4/4 - 0s - loss: 88.1538 - accuracy: 0.4643 - 154ms/epoch - 38ms/step
Epoch 3/5
4/4 - 0s - loss: 5.2738 - accuracy: 0.5893 - 154ms/epoch - 39ms/step
Epoch 4/5
4/4 - 0s - loss: 0.4535 - accuracy: 0.8036 - 156ms/epoch - 39ms/step
Epoch 5/5
4/4 - 0s - loss: 0.3553 - accuracy: 0.8929 - 148ms/epoch - 37ms/step
4/4 [==============================] - 0s 20ms/step - loss: 0.2683 - accuracy: 0.9821
Epoch 1/5
4/4 - 1s - loss: 193.0846 - accuracy: 0.5179 - 751ms/epoch - 188ms/step
Epoch 2/5
4/4 - 0s - loss: 4.4712 - accuracy: 0.5446 - 241ms/epoch - 60ms/step
Epoch 3/5
4/4 - 0s - loss: 0.5727 - accuracy: 0.6250 - 241ms/epoch - 60ms/step
Epoch 4/5
4/4 - 0s - loss: 0.4960 - accuracy: 0.7054 - 244ms/epoch - 61ms/step
Epoch 5/5
4/4 - 0s - loss: 0.4506 - accuracy: 0.7500 - 235ms/epoch - 59ms/step
4/4 [==============================] - 0s 29ms/step - loss: 0.3831 - accuracy: 0.7411
Epoch 1/5
4/4 - 1s - loss: 1035

...

 67%|██████▋   | 2/3 [00:14<00:07,  7.04s/it]

Epoch 1/5
4/4 - 1s - loss: 227.9049 - accuracy: 0.4911 - 737ms/epoch - 184ms/step
Epoch 2/5
4/4 - 0s - loss: 13.7424 - accuracy: 0.5625 - 244ms/epoch - 61ms/step
Epoch 3/5
4/4 - 0s - loss: 0.9894 - accuracy: 0.4911 - 244ms/epoch - 61ms/step
Epoch 4/5
4/4 - 0s - loss: 0.6932 - accuracy: 0.4911 - 240ms/epoch - 60ms/step
Epoch 5/5
4/4 - 0s - loss: 0.6932 - accuracy: 0.4911 - 234ms/epoch - 58ms/step
4/4 [==============================] - 0s 28ms/step - loss: 0.6932 - accuracy: 0.4911
Epoch 1/5
4/4 - 1s - loss: 165.0105 - accuracy: 0.5089 - 748ms/epoch - 187ms/step
Epoch 2/5
4/4 - 0s - loss: 8.2626 - accuracy: 0.5357 - 242ms/epoch - 60ms/step
Epoch 3/5
4/4 - 0s - loss: 0.8436 - accuracy: 0.6875 - 241ms/epoch - 60ms/step
Epoch 4/5
4/4 - 0s - loss: 0.5732 - accuracy: 0.7411 - 244ms/epoch - 61ms/step
Epoch 5/5
4/4 - 0s - loss: 0.5566 - accuracy: 0.7232 - 234ms/epoch - 58ms/step
4/4 [==============================] - 0s 29ms/step - loss: 0.5303 - accuracy: 0.6161
Epoch 1/5
4/4 - 1s - loss: 384.


 accuracy 0.9196428656578064
start_point = {'keras.compile.0': {'loss': ['binary_crossentropy'], 'optimizer': ['adam']}, 'keras.fit.0': {'epochs': [5], 'batch_size': [32], 'verbose': [2]}, 'keras.layers.Conv2D.1': {'filters': [64], 'kernel_size': [(3, 3)], 'activation': ['relu'], 'input_shape': [(256, 256, 3)]}, 'keras.layers.MaxPooling2D.2': {'pool_size': [(2, 2)]}, 'keras.layers.Conv2D.3': {'filters': [16], 'kernel_size': [(3, 3)], 'activation': ['relu']}, 'keras.layers.MaxPooling2D.4': {'pool_size': [(2, 2)]}, 'keras.layers.Flatten.5': {}, 'keras.layers.Dense.6': {'units': [50], 'activation': ['relu']}, 'keras.layers.Dropout.7': {'rate': [0.6000000000000001]}, 'keras.layers.Dense.8': {'units': [2], 'activation': ['softmax']}}
1/4 [======>.......................] - ETA: 0s - loss: 0.6933 - accuracy: 0.4375

4/4 [==============================] - 0s 63ms/step - loss: 0.6932 - accuracy: 0.4911
time: 23.212714433670044


1/1 [==============================] - 0s 50ms/step - loss: 0.6931 - accuracy: 0.5357
test score: 0.5357142686843872
